In [1]:
# xgboost
#Insurance policy and those who qualify for it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importing data sets
dataset_train = pd.read_csv("train_data.csv")
dataset_test = pd.read_csv("test_data.csv")

In [3]:
dataset_train.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [4]:
dataset_train.shape

(7160, 14)

In [5]:
dataset_train.isnull().sum()

Customer Id             0
YearOfObservation       0
Insured_Period          0
Residential             0
Building_Painted        0
Building_Fenced         0
Garden                  7
Settlement              0
Building Dimension    106
Building_Type           0
Date_of_Occupancy     508
NumberOfWindows         0
Geo_Code              102
Claim                   0
dtype: int64

In [6]:
dataset_train.columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy',
       'NumberOfWindows', 'Geo_Code', 'Claim'],
      dtype='object')

In [7]:
X_train = dataset_train.iloc[:, 1:11].values
y_train = dataset_train.iloc[:, 13].values
X_test = dataset_test.iloc[:, 1:11].values

In [8]:
# Fixing the missing data, using mean of each column.
from sklearn.impute import SimpleImputer

#WORKING ON THE DUMMY VARIABLE - MISSING VALUE
missingvalues_1 = SimpleImputer(missing_values=np.nan, strategy="most_frequent", verbose=0)
missingvalues_train_1 = missingvalues_1.fit(X_train[:,[5]])
rows_3 =np.array(missingvalues_train_1.transform(X_train[:,[5]]))

missingvalues = SimpleImputer(missing_values=np.nan, strategy="mean", verbose=0)
# X_train missing values
missingvalues_train = missingvalues.fit(X_train[:,[7, 9]])
rows =np.array(missingvalues_train.transform(X_train[:,[7,9]]))


row_1 = rows[:,0]
row_2 = rows[:,[1]]

In [9]:
# Removing the columns that was having the missing values
column = ['YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy']
df = pd.DataFrame(X_train)#, columns= column)
# Adding the columns back
df[5]=rows_3
df[7]= row_1
df[9]=row_2
X_train = df

In [10]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9
0,2013,1.0,0,N,V,V,U,290.0,1,1960.0
1,2015,1.0,0,V,N,O,R,490.0,1,1850.0
2,2014,1.0,0,N,V,V,U,595.0,1,1960.0
3,2013,1.0,0,V,V,V,U,2840.0,1,1960.0
4,2014,1.0,0,V,N,O,R,680.0,1,1800.0


In [11]:
#X_train = pd.DataFrame(X_train,columns=column)

In [14]:
X_train

,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# X_test missing values
missingvalues_test_1 = SimpleImputer(missing_values=np.nan, strategy="most_frequent", verbose=0)
missingvalues_test_1 = missingvalues_1.fit(X_test[:,[5]])
test_row_3 =np.array(missingvalues_test_1.transform(X_test[:,[5]]))

missingvalues = SimpleImputer(missing_values=np.nan, strategy="mean", verbose=0)
missingvalues_test = missingvalues.fit(X_test[:,[9]]) 
test_row = np.array(missingvalues_test.transform(X_test[:,[9]]))

# Removing the column that was having the missing values
df_test = pd.DataFrame(X_test)
# Adding the columns back
df_test[5]= test_row_3
df_test[9]= test_row
X_test = df_test

In [9]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9
0,2013,1.0,0,V,N,O,R,300.0,1,1960.0
1,2016,0.997268,0,V,N,O,R,300.0,1,1960.0
2,2013,0.369863,0,V,V,V,U,790.0,1,1960.0
3,2014,1.0,0,V,N,O,R,1405.0,1,2004.0
4,2016,1.0,0,V,N,O,R,1405.0,1,2004.0


In [9]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [15]:
#PART 2
import xgboost as xgb
from xgboost import XGBClassifier 
classifier = XGBClassifier(use_label_encoder=False)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

[11:53:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
y_pred_prob = classifier.predict_proba(X_test)
#from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_test,y_pred)

In [17]:
import joblib
model_file = open("models/xgb_model.pkl","wb")
joblib.dump(classifier,model_file)
model_file.close()

In [18]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X= X_train, y = y_train, cv=10) 
accuracies.mean()

[11:53:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:53:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:53:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:53:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.759217877094972

In [14]:
accuracies.std()

0.03248640579677844